<a href="https://www.kaggle.com/code/esracum/lgg-mri-segmentation?scriptVersionId=291071930" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# U-Net nedir? 


U-Net, özellikle tıbbi görüntüleme ve segmentasyon görevlerinde etkili olan bir derin öğrenme mimarisidir. 2015 yılında Olaf Ronneberger ve arkadaşları tarafından tanıtılmıştır ve temel olarak iki aşamalı bir yapıdan oluşur: encoder ve decoder.

# U-Net Mimarisinin Temel Özellikleri?

In [ ]:
#     [Input Image]
#           ↓
#     [Encoder Path]
#           ↓
#     [Bottleneck]
#           ↓
#     [Decoder Path]
#           ↓
#     [Output Image]


U-Net Mimarisinin Temel Özellikleri:
Enkoder (Encoder) Bölümü:

Konvolüsyonel Katmanlar: Girdi görüntüsünden özellikleri çıkarmak için bir dizi konvolüsyonel katman kullanır.
Max Pooling: Özellik haritasının boyutlarını küçültmek ve önemli özellikleri öğrenmek için kullanılır. Bu işlem, özelliklerin genel bir temsiline geçiş yapar.
Dekoder (Decoder) Bölümü:

Up-sampling: Özellik haritalarını orijinal boyutlarına geri getirmek için up-sampling (büyütme) işlemi yapar.
Konvolüsyonel Katmanlar: Up-sampling işlemi sonrasında elde edilen özellik haritalarını daha ayrıntılı hale getirir.
Skip Connections: Enkoder bölümündeki özellik haritalarını dekoder bölümüne doğrudan aktarır. Bu, hem yüksek düzeyde hem de ayrıntılı özelliklerin kullanılmasını sağlar ve segmentasyon sonucunun daha hassas olmasına yardımcı olur.
Sonuç:

Son Konvolüsyon Katmanı: Sonuç olarak, çıktı sınıf sayısına göre bir konvolüsyon katmanı eklenir. Bu katman, segmentasyon sonucunu üretir.


# U-Net Mimarisinin Avantajları?

Ayrıntılı Segmentasyon: Skip connections, modelin hem büyük yapıları hem de ince detayları öğrenmesine yardımcı olur, bu da ayrıntılı segmentasyon sonuçları sağlar.
Etkili Eğitim: U-Net, özellikle sınırlı veri miktarıyla etkili bir şekilde çalışabilir, çünkü skip connections daha az veriyle modelin performansını iyileştirir.
İyi Performans: Tıbbi görüntüleme gibi yüksek hassasiyet gerektiren görevlerde etkili bir performans sergiler.

# U-Net'in Kullanım Alanları

Tıbbi Görüntüleme: MR, CT taramaları gibi tıbbi görüntülerde organ segmentasyonu ve lezyon tespiti.
Görüntü Segmentasyonu: Genel görüntü segmentasyonu görevlerinde kullanılır.
Görüntü İyileştirme: Görüntülerin düzeltilmesi veya iyileştirilmesi görevlerinde de uygulanabilir.

# Gerekli Kütüphanelerin İmport Edilmesi

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import backend as K

plt.style.use("ggplot")


In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

import cv2
from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
from skimage.color import rgb2gray
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



In [ ]:
# Setting size parameters of images
im_width = 256
im_height = 256

In [ ]:
image_filenames_train = []

# creating a list of all files containing the word 'mask'
# in Local Machine
# mask_files = glob('../../input/lgg-mri-segmentation/kaggle_3m/*/*_mask*')

# in Kaggle
mask_files = glob('../input/lgg-mri-segmentation/kaggle_3m/*/*_mask*')

for i in mask_files:
    image_filenames_train.append(i.replace('_mask', ''))

print(image_filenames_train[:10])
len(image_filenames_train)

# Görüntü ve Maskeleri Görselleştirme Fonksiyonu

In [ ]:
def plot_from_img_path(rows, columns, list_img_path, list_mask_path):
    """ 
    Bu fonksiyon, verilen görüntü ve maske yollarından bir grid (ızgara) oluşturur ve bunları görüntüler.
    Görüntüleri okumak ve işlemek için OpenCV kullanır, plot işlemi için matplotlib kullanılır.

    Parametreler:
    -----------
    rows : int
        Grid'in satır sayısı.

    columns : int
        Grid'in sütun sayısı.

    list_img_path : list of str
        Görüntü dosyalarının yolunu içeren liste.

    list_mask_path : list of str
        Maske dosyalarının yolunu içeren liste.
    """
    fig = plt.figure(figsize=(12, 12))
    for i in range(1, rows * columns + 1):
        fig.add_subplot(rows, columns, i)
        img_path = list_img_path[i]
        mask_path = list_mask_path[i]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path)
        plt.imshow(image)
        plt.imshow(mask, alpha=0.3)
    plt.show()


# Dice Katsayısı Hesaplama Fonksiyonu

* Dice Katsayısı: Bu fonksiyon, görüntü segmentasyonunda kullanılan bir ölçüt olan Dice katsayısını hesaplar. Bu katsayı, tahmin edilen ve gerçek değerler arasındaki örtüşme miktarını gösterir.

In [ ]:
def dice_coefficients(y_true, y_pred, smooth=100):
    """
    Gerçek ve tahmin edilen değerler arasındaki Dice benzerlik katsayısını hesaplar.

    Dice katsayısı, iki örnek arasındaki benzerliği ölçmek için kullanılan bir metriktir. 
    0 ile 1 arasında bir değer alır; 1, iki örneğin tamamen aynı olduğunu, 0 ise hiç örtüşmediklerini belirtir.

    Parametreler:
    -----------
    y_true : tensor
        Gerçek değerler (genellikle bir ikili maske).

    y_pred : tensor
        Tahmin edilen değerler (genellikle bir ikili maske).

    smooth : float, optional
        Sıfır bölmeyi önlemek için kullanılan bir düzeltme faktörü. Varsayılan değer 100'dür.
    
    Döndürülen:
    --------
    float
        Gerçek ve tahmin edilen değerler arasındaki Dice benzerlik katsayısı.
    """
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)


# Dice Kayıp Fonksiyonu

* Dice Kaybı: Bu fonksiyon, segmentasyon modelleri için Dice kaybını hesaplar. Kayıp fonksiyonu olarak negatif Dice katsayısı kullanılır, çünkü optimizasyon algoritmaları genellikle minimize etmek için tasarlanmıştır.

In [ ]:
def dice_coefficients_loss(y_true, y_pred, smooth=100):
    """
    Görüntü segmentasyon modelleri için Dice kaybı fonksiyonu.

    Dice kaybı, tahmin edilen (y_pred) ve gerçek (y_true) maskeler arasındaki örtüşmeyi ölçer.
    Optimizasyon algoritmaları genellikle bir fonksiyonu minimize etmek için tasarlandığından,
    Dice katsayısının negatifini alırız, böylece optimizasyon süreci bu kaybı minimize etmeye çalışır.

    Parametreler:
    -----------
    y_true : tensor
        Gerçek değerler (genellikle bir ikili maske).

    y_pred : tensor
        Tahmin edilen değerler (genellikle bir ikili maske).

    smooth : float, optional
        Sıfır bölmeyi önlemek için kullanılan bir düzeltme faktörü. Varsayılan değer 100'dür.
    
    Döndürülen:
    --------
    float
        Hesaplanan Dice kaybı değeri.
    """
    return -dice_coefficients(y_true, y_pred, smooth)


# Intersection over Union (IoU) Hesaplama Fonksiyonu

* IoU: Bu fonksiyon, tahmin edilen ve gerçek maskelerin ne kadar örtüştüğünü gösteren Intersection over Union (IoU) değerini hesaplar.

In [ ]:
def iou(y_true, y_pred, smooth=100):
    """
    Gerçek ve tahmin edilen değerler arasındaki Intersection over Union (IoU) değerini hesaplar.

    IoU, segmentasyon problemlerinde tahmin edilen maskelerin gerçek maskelerle ne kadar örtüştüğünü ölçen bir metriktir.

    Parametreler:
    -----------
    y_true : tensor
        Gerçek değerler (genellikle bir ikili maske).

    y_pred : tensor
        Tahmin edilen değerler (genellikle bir ikili maske).

    smooth : float, optional
        Sıfır bölmeyi önlemek için kullanılan bir düzeltme faktörü. Varsayılan değer 100'dür.
    
    Döndürülen:
    --------
    float
        Gerçek ve tahmin edilen değerler arasındaki IoU değeri.
    """
    intersection = K.sum(y_true * y_pred)
    sum = K.sum(y_true + y_pred)
    iou = (intersection + smooth) / (sum - intersection + smooth)
    return iou


#  Jaccard Mesafesi Hesaplama Fonksiyonu

* Jaccard Mesafesi: Bu fonksiyon, tahmin edilen ve gerçek etiketler arasındaki Jaccard mesafesini hesaplar. Jaccard mesafesi, setler arasındaki benzerliği ölçen bir metriktir ve daha düşük bir değer, daha iyi bir performans gösterir.

In [ ]:
def jaccard_distance(y_true, y_pred):
    """
    Gerçek etiketlerle tahmin edilen etiketler arasındaki Jaccard mesafesini hesaplar.

    Jaccard mesafesi, setler arasındaki benzerliği ölçen bir metriktir. 
    Tahmin edilen etiketler ve gerçek etiketler arasındaki Jaccard mesafesi ne kadar düşükse,
    modelin performansı o kadar iyidir.

    Parametreler:
    -----------
    y_true : np.array
        Gerçek etiket (ikili maske) dizisi.

    y_pred : np.array
        Tahmin edilen etiket (ikili maske) dizisi.
    
    Döndürülen:
    --------
    float
        Gerçek etiketlerle tahmin edilen etiketler arasındaki Jaccard mesafesi.
    """
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)
    return -iou(y_true_flatten, y_pred_flatten)


In [ ]:
plot_from_img_path(3, 3 , image_filenames_train, mask_files )

# DataFrame Oluşturmak ve Veriyi Train, Accuracy ve Test Setlerine Ayırma

In [ ]:
df = pd.DataFrame(data={'image_filenames_train': image_filenames_train, 'mask': mask_files })

df_train, df_test = train_test_split(df, test_size=0.1)

# Further split this val and train
df_train, df_val = train_test_split(df_train, test_size=0.2)

print(df_train.shape)
print(df_test.shape)
print(df_val.shape)

In [ ]:
def train_generator(
    data_frame,
    batch_size,
    augmentation_dict,
    image_color_mode="rgb",
    mask_color_mode="grayscale",
    image_save_prefix="image",
    mask_save_prefix="mask",
    save_to_dir=None,
    target_size=(256, 256),
    seed=1,
):
    """
    Bu fonksiyon, görüntü ve maskeleri aynı anda üretebilen bir veri üreticisi (generator) oluşturur.
    `image_datagen` ve `mask_datagen` için aynı seed kullanarak, görüntü ve maskenin aynı dönüşümleri
    geçmesini sağlar. Eğer üretici sonuçlarını görselleştirmek isterseniz, `save_to_dir` parametresini
    belirli bir dizinle ayarlayın.
    """
    image_datagen = ImageDataGenerator(**augmentation_dict)
    mask_datagen = ImageDataGenerator(**augmentation_dict)

    image_generator = image_datagen.flow_from_dataframe(
        data_frame,
        x_col="image_filenames_train",  # Görüntü dosyalarının isimlerini içeren sütun adı
        class_mode=None,
        color_mode=image_color_mode,  # Görüntülerin renk modu
        target_size=target_size,  # Görüntülerin hedef boyutu
        batch_size=batch_size,  # Her bir seferde kaç görüntü üretileceği
        save_to_dir=save_to_dir,  # Görüntülerin kaydedileceği dizin
        save_prefix=image_save_prefix,  # Görüntü dosya isimlerinin ön eki
        seed=seed,  # Rastgelelik için seed değeri
    )

    mask_generator = mask_datagen.flow_from_dataframe(
        data_frame,
        x_col="mask",  # Maske dosyalarının isimlerini içeren sütun adı
        class_mode=None,
        color_mode=mask_color_mode,  # Maskelerin renk modu (genellikle gri tonlama)
        target_size=target_size,  # Maskelerin hedef boyutu
        batch_size=batch_size,  # Her bir seferde kaç maske üretileceği
        save_to_dir=save_to_dir,  # Maskelerin kaydedileceği dizin
        save_prefix=mask_save_prefix,  # Maske dosya isimlerinin ön eki
        seed=seed,  # Rastgelelik için seed değeri
    )

    train_gen = zip(image_generator, mask_generator)
    
    # Görüntü normalizasyonu ve teşhis sonrası nihai dönüş
    for (img, mask) in train_gen:
        img, mask = normalize_and_diagnose(img, mask)
        yield (img, mask)


# normalize_and_diagnose Fonksiyonu:

* Görüntü ve maskeleri 0 ile 1 arasına normalize eder.
* Maske değerlerini, 0.5'ten büyük olanları 1 ve 0.5 veya daha küçük olanları 0 olarak ayarlar. Bu, maskelerin belirli bir eşik değerine göre ikili hale gelmesini sağlar.
*Maskenin 0.5'ten küçük değerleri, tümör içermediği düşünülen tamamen siyah bölgeler olarak kabul edilir.

In [ ]:
def normalize_and_diagnose(img, mask):
    """
    Görüntü ve maskeyi normalleştiren ve teşhis eden fonksiyon.
    Maske normalizasyonundan sonra değer <= 0.5 olan maskeler tamamen siyah olarak kabul edilir
    ve tümör içerdiği düşünülmez.
    """
    img = img / 255  # Görüntüyü 0-1 aralığına normalize et
    mask = mask / 255  # Maskeyi 0-1 aralığına normalize et
    mask[mask > 0.5] = 1  # Maskedeki değerler 0.5'ten büyükse 1 olarak ayarla
    mask[mask <= 0.5] = 0  # Maskedeki değerler 0.5 veya daha küçükse 0 olarak ayarla
    return(img, mask)


In [ ]:
EPOCHS = 25
BATCH_SIZE = 32
learning_rate = 1e-3
smooth=100

# U-Net Modeli 

In [ ]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import (
    Input,
    Activation,
    BatchNormalization,
    Dropout,
    Lambda,
    Conv2D,
    Conv2DTranspose,
    MaxPooling2D,
    concatenate,
)
from tensorflow.keras import backend as K


def unet(input_size=(256, 256, 3)):
    """
    This function creates and returns a U-Net model. U-Net is a type of convolutional neural network
    designed for fast and precise segmentation of images. It consists of a contracting (downsampling)
    path and an expansive (upsampling) path, which gives it a U-shaped architecture.

    Parameters:
    -----------
    input_size : tuple of int
        The size of the input images. It is a 3-tuple for (height, width, channels).
        Default is (256, 256, 3).

    Returns:
    --------
    model : keras.models.Model
        The constructed U-Net model.
    """
    inputs = Input(input_size)

    # First DownConvolution / Encoder Leg will begin, so start with Conv2D
    conv1 = Conv2D(filters=64, kernel_size=(3, 3), padding="same")(inputs)
    bn1 = Activation("relu")(conv1)
    conv1 = Conv2D(filters=64, kernel_size=(3, 3), padding="same")(bn1)
    bn1 = BatchNormalization(axis=3)(conv1)
    bn1 = Activation("relu")(bn1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(bn1)

    conv2 = Conv2D(filters=128, kernel_size=(3, 3), padding="same")(pool1)
    bn2 = Activation("relu")(conv2)
    conv2 = Conv2D(filters=128, kernel_size=(3, 3), padding="same")(bn2)
    bn2 = BatchNormalization(axis=3)(conv2)
    bn2 = Activation("relu")(bn2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(bn2)

    conv3 = Conv2D(filters=256, kernel_size=(3, 3), padding="same")(pool2)
    bn3 = Activation("relu")(conv3)
    conv3 = Conv2D(filters=256, kernel_size=(3, 3), padding="same")(bn3)
    bn3 = BatchNormalization(axis=3)(conv3)
    bn3 = Activation("relu")(bn3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)

    conv4 = Conv2D(filters=512, kernel_size=(3, 3), padding="same")(pool3)
    bn4 = Activation("relu")(conv4)
    conv4 = Conv2D(filters=512, kernel_size=(3, 3), padding="same")(bn4)
    bn4 = BatchNormalization(axis=3)(conv4)
    bn4 = Activation("relu")(bn4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(bn4)

    conv5 = Conv2D(filters=1024, kernel_size=(3, 3), padding="same")(pool4)
    bn5 = Activation("relu")(conv5)
    conv5 = Conv2D(filters=1024, kernel_size=(3, 3), padding="same")(bn5)
    bn5 = BatchNormalization(axis=3)(conv5)
    bn5 = Activation("relu")(bn5)

    """ Now UpConvolution / Decoder Leg will begin, so start with Conv2DTranspose
    The gray arrows (in the above image) indicate the skip connections that concatenate the encoder feature map with the decoder, which helps the backward flow of gradients for improved training. """
    up6 = concatenate(
        [
            Conv2DTranspose(512, kernel_size=(2, 2), strides=(2, 2), padding="same")(
                bn5
            ),
            conv4,
        ],
        axis=3,
    )
    """ After every concatenation we again apply two consecutive regular convolutions so that the model can learn to assemble a more precise output """
    conv6 = Conv2D(filters=512, kernel_size=(3, 3), padding="same")(up6)
    bn6 = Activation("relu")(conv6)
    conv6 = Conv2D(filters=512, kernel_size=(3, 3), padding="same")(bn6)
    bn6 = BatchNormalization(axis=3)(conv6)
    bn6 = Activation("relu")(bn6)

    up7 = concatenate(
        [
            Conv2DTranspose(256, kernel_size=(2, 2), strides=(2, 2), padding="same")(
                bn6
            ),
            conv3,
        ],
        axis=3,
    )
    conv7 = Conv2D(filters=256, kernel_size=(3, 3), padding="same")(up7)
    bn7 = Activation("relu")(conv7)
    conv7 = Conv2D(filters=256, kernel_size=(3, 3), padding="same")(bn7)
    bn7 = BatchNormalization(axis=3)(conv7)
    bn7 = Activation("relu")(bn7)

    up8 = concatenate(
        [
            Conv2DTranspose(128, kernel_size=(2, 2), strides=(2, 2), padding="same")(
                bn7
            ),
            conv2,
        ],
        axis=3,
    )
    conv8 = Conv2D(filters=128, kernel_size=(3, 3), padding="same")(up8)
    bn8 = Activation("relu")(conv8)
    conv8 = Conv2D(filters=128, kernel_size=(3, 3), padding="same")(bn8)
    bn8 = BatchNormalization(axis=3)(conv8)
    bn8 = Activation("relu")(bn8)

    up9 = concatenate(
        [
            Conv2DTranspose(64, kernel_size=(2, 2), strides=(2, 2), padding="same")(
                bn8
            ),
            conv1,
        ],
        axis=3,
    )
    conv9 = Conv2D(filters=64, kernel_size=(3, 3), padding="same")(up9)
    bn9 = Activation("relu")(conv9)
    conv9 = Conv2D(filters=64, kernel_size=(3, 3), padding="same")(bn9)
    bn9 = BatchNormalization(axis=3)(conv9)
    bn9 = Activation("relu")(bn9)

    conv10 = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(bn9)

    return Model(inputs=[inputs], outputs=[conv10])

In [ ]:
model = unet()
model.summary

In [ ]:
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    return lr * decay_rate

lr_scheduler = LearningRateScheduler(scheduler)


train_generator_param = dict(rotation_range=0.2,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest')

train_gen = train_generator(df_train, BATCH_SIZE,
                                train_generator_param,
                                target_size=(im_height, im_width))
    
test_gen = train_generator(df_val, BATCH_SIZE,
                                dict(),
                                target_size=(im_height, im_width))
    
model = unet(input_size=(im_height, im_width, 3))


# decay kullanmadan learning_rate doğrudan belirleyin

opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-7, amsgrad=False)

decay_rate = learning_rate / EPOCHS

model.compile(optimizer=opt, loss=dice_coefficients_loss, metrics=["binary_accuracy", iou, dice_coefficients])

callbacks = [ModelCheckpoint('unet.keras', verbose=1, save_best_only=True)]




history = model.fit(train_gen,
                    steps_per_epoch=int(len(df_train) / BATCH_SIZE), 
                    epochs=EPOCHS, 
                    callbacks=callbacks,
                    validation_data = test_gen,
                    validation_steps=int(len(df_val) / BATCH_SIZE))

In [ ]:
history_post_training = history.history

train_dice_coeff_list = history_post_training['dice_coefficients']
test_dice_coeff_list = history_post_training['val_dice_coefficients']

train_jaccard_list = history_post_training['iou']
test_jaccard_list = history_post_training['val_iou']

train_loss_list = history_post_training['loss']
test_loss_list = history_post_training['val_loss']

plt.figure(1)
plt.plot(test_loss_list, 'b-')
plt.plot(train_loss_list, 'r-')

plt.xlabel('iterations')
plt.ylabel('loss')
plt.title('loss graph', fontsize=12)

plt.figure(2)
plt.plot(train_dice_coeff_list, 'b-')
plt.plot(test_dice_coeff_list, 'r-')

plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.title('Accuracy graph', fontsize=12)
plt.show()

In [ ]:
print(history.history.keys())


In [ ]:
import random
for i in range(20):
    index = np.random.randint(1, len(df_test.index))
    img = cv2.imread(df_test['image_filenames_train'].iloc[index])
    img = cv2.resize(img, (im_height, im_width))
    img = img/255
    # print(imgs.shape) (256, 256 , 3)
    img = img[np.newaxis, :, :, : ]
    # print(img.shape) # (1, 256, 256, 3)

    predicted_img = model.predict(img)
    
    plt.figure(figsize=(12, 12))
    plt.subplot(1, 3, 1)
    plt.imshow(np.squeeze(img))
    plt.title('Original Image')

    plt.subplot(1, 3, 2)
    plt.imshow(np.squeeze(cv2.imread(df_test['mask'].iloc[index])))
    plt.title('Original Mask')

    plt.subplot(1, 3, 3)
    plt.imshow(np.squeeze(predicted_img) > 0.5 )
    plt.title('Prediction')
    plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Eğitim ve Doğrulama Kayıp Grafiği
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Eğitim ve Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()

# Eğitim ve Doğrulama Doğruluk Grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['binary_accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_binary_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()

plt.show()

# Eğitim ve Doğrulama IoU Grafiği
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['iou'], label='Eğitim IoU')
plt.plot(history.history['val_iou'], label='Doğrulama IoU')
plt.title('Eğitim ve Doğrulama IoU')
plt.xlabel('Epoch')
plt.ylabel('IoU')
plt.legend()

# Eğitim ve Doğrulama Dice Coefficient Grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['dice_coefficients'], label='Eğitim Dice Coefficient')
plt.plot(history.history['val_dice_coefficients'], label='Doğrulama Dice Coefficient')
plt.title('Eğitim ve Doğrulama Dice Coefficient')
plt.xlabel('Epoch')
plt.ylabel('Dice Coefficient')
plt.legend()

plt.show()


In [ ]:
# Kullanılan metrikleri listele
print("Kullanılan Metrikler: ", model.metrics_names)

# Kullanılan kayıp fonksiyonu
print("Kayıp Fonksiyonu: ", model.loss)


In [ ]:
model.save('/kaggle/working/model.h5')



In [ ]:
# Test veri seti üzerindeki performansı değerlendirin
results = model.evaluate(test_gen, steps=int(len(df_val) / BATCH_SIZE))

# Döndürülen tüm değerleri yazdırın
print(f"Test Results: {results}")


In [ ]:
#faydalanılan kaynak:https://www.youtube.com/watch?v=X1q993i7Gic&list=PLxqBkZuBynVRAcsIU-aKf46keJvnV3z5n